In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import getcwd
from scipy.constants import e, k

from src.utils.common import read_file, write_file, get_data, print_progress_bar, rmse
from src.classes.SolarCell import SolarCell


inp_dir = getcwd() + '/input_files/'

temp = 25
T = temp + 273
v_t = k * T / e

In [ ]:
data = read_file(inp_dir + 'diode_iv_curve.txt')
v = np.array([val for i, val in enumerate(data) if not i % 2])
i = np.array([val for i, val in enumerate(data) if i % 2])

write_file(inp_dir + 'diode_i_curve.txt', i)
write_file(inp_dir + 'diode_u_curve.txt', v)

In [ ]:
zero_cur = init_cur = i[0] / np.exp(v[0] / v_t - 1)
target_error = 5
errors = []


while True:
    eta, error = minimize(init_cur)
    init_cur = i[0] / (np.exp(v[0] / v_t / eta) - 1)
    metric = abs(init_cur - zero_cur) / zero_cur * 100
    # print(init_cur, zero_cur, metric)
    
    
    if metric < target_error:
        break
        
print(init_cur, eta)

In [ ]:
steps = 100
errors = []
min_error = 10 ** 10
x_fit_vals = v[-12:]
y_fit_vals = i[-12:]

init_cur = i[0] / (np.exp(v[0] / v_t) - 1)
fit = np.polyfit(x_fit_vals, y_fit_vals, 1)
init_rs = 1 / fit[0]


for idx, eta in enumerate(np.linspace(1, 2, steps)):
    print_progress_bar(idx + 1, steps)
    for cur in np.linspace(0.75 * init_cur, 1.25 * init_cur, steps):
        for rs in np.linspace(0.75 * init_rs, 1.25 * init_rs, steps):
            approx = cur * (np.exp((v - rs * i) / v_t / eta) - 1)
            error = rmse(i, approx)
            errors.append(error)
    
            if error < min_error:
                min_error = error
                min_eta, min_cur, min_rs = eta, cur, rs
                
print(min_eta, min_cur, min_rs)

In [ ]:
eps = 10e-2
eta = 0.1

a1 = min_cur * (np.exp((v - min_rs * i) / v_t / min_eta) - 1)

ii = [0]
vv = np.linspace(0, v[-1], 100)

for idx, v_val in enumerate(vv[1:]):    
    old_c = ii[-1]
    new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
    
    while abs(new_c - old_c) > eps:
        old_c = new_c
        new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
        new_c = old_c + eta * (new_c - old_c)
    
    ii.append(new_c)


plt.plot(-vv, ii)
plt.plot(v, i)
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('I-V Curve of Diode')
plt.grid(True)
plt.show()

In [ ]:
solar_cell_num = 54
G = 800

r_file = open(inp_dir + "KC200GT_new.txt")
e_voltage, e_current = get_data(r_file)
e_voltage = np.array(e_voltage)
r_file.close()

My_one_diode_fa_KC200GT = SolarCell(cell_num=54, Voc=32.9, Isc=8.21,
                       Pmpp=200, Impp=7.61, Vmpp=26.3,
                       Ku=-0.123, Ki=3.18 / 1000, a1=1.0, a2=10**10,
                       thermal_coefficient_type='absolute', G=1000,
                       experimental_V=e_voltage,
                       experimental_I=e_current,
                       is_experimental_data_provided=True,
                       mode='overall')

My_one_diode_fa_KC200GT.end_fit_points_number

In [ ]:
My_one_diode_fa_KC200GT.run()

In [ ]:
v_rev = -vv
i_rev = My_one_diode_fa_KC200GT.fixed_point_method_rv(My_one_diode_fa_KC200GT.Rs, 
                                                      My_one_diode_fa_KC200GT.Rp, v_rev)[1:]
i_rev_diode = np.sum([i_rev, ii], axis=0)


i_for = My_one_diode_fa_KC200GT.fixed_point_method_rv(My_one_diode_fa_KC200GT.Rs, 
                                                      My_one_diode_fa_KC200GT.Rp, My_one_diode_fa_KC200GT.V_mesh)[1:]

In [ ]:
aa = np.array(My_one_diode_fa_KC200GT.V_mesh) * 300 / 1000

In [ ]:
plt.plot(My_one_diode_fa_KC200GT.e_V, My_one_diode_fa_KC200GT.e_I)
plt.plot(My_one_diode_fa_KC200GT.V_mesh, i_for)
# plt.plot(v_rev, i_rev)
# plt.plot(v_rev, i_rev_diode)
plt.grid()

In [ ]:
solar_cell_num = 54
G = 800

KC200GT_1000 = SolarCell(cell_num=54, Voc=32.9, Isc=8.21, a1=1.0, a2=1.2,
                         thermal_coefficient_type='absolute', G=1000,
                         experimental_V=e_voltage,
                         experimental_I=e_current,
                         is_experimental_data_provided=True,
                         mode='overall')

KC200GT_800 = SolarCell_new(cell_num=54, Voc=32.9, Isc=8.21,
                         thermal_coefficient_type='absolute', G=1000,
                         experimental_V=e_voltage,
                         experimental_I=e_current,
                         is_experimental_data_provided=True,
                         mode='overall')

In [ ]:
KC200GT_1000.run()

In [ ]:
KC200GT_800.run()

In [ ]:
KC200GT_800.Ipv = KC200GT_800.Ipv * 800 / 1000
KC200GT_800.Ipv

In [ ]:
cur = KC200GT_800.find_best_fit_3(0.26297693697173036, 120.35692260028858, 800)
cur = cur[0]

In [ ]:
cur_1 = KC200GT_800.fixed_point_method_rv(0.26297693697173036, 120.35692260028858, KC200GT_800.V_mesh)
cur_1 = [i for i in cur_1 if i >= 0]

In [ ]:
plt.plot(KC200GT_800.V_mesh, cur[0])
plt.plot(KC200GT_800.V_mesh, KC200GT_800.e_I)
plt.grid()

In [ ]:
class SolarCell_new:
    def __init__(self,
                 cell_num: int = 36,
                 mesh_points_num: int = 1000,

                 Voc: float = 22.0,
                 Isc: float = 3.45,

                 Pmpp: float = 55,
                 Impp: float = 3.15,
                 Vmpp: float = 17.4,

                 experimental_V: Sequence[int] = (),
                 experimental_I: Sequence[Union[float, int]] = (),

                 Ku: Union[float, int] = 1.0,
                 Ki: Union[float, int] = 1.0,
                 Kp: Union[float, int] = 1.0,

                 T: Union[float, int] = 25,
                 Tstc: Union[float, int] = 25,

                 G: Union[float, int] = 1000,
                 Gstc: Union[float, int] = 1000,

                 custom_thermal_voltage_func=None,
                 custom_photovoltaic_current_func=None,
                 custom_backward_current_func=None,

                 thermal_coefficient_type="relative",
                 is_experimental_data_provided=False,

                 a1=1.0, a2=1.2,

                 I0=None, Ipv=None,
                 start_slope=None,
                 end_slope=None,
                 Vt=None,

                 fixed_point_method_tolerance=10 * 10 ** -12,
                 brute_force_steps=50,
                 mode="overall",
                 approx_range_minimization=None
                 ):

        if thermal_coefficient_type in ("relative", "absolute"):
            self.thermal_coefficient_type = thermal_coefficient_type
        else:
            raise ValueError("Unknown type of 'thermal_coefficient_type' parameter")

        # Custom functions
        self.custom_thermal_voltage_func = custom_thermal_voltage_func
        self.custom_photovoltaic_current_func = custom_photovoltaic_current_func
        self.custom_backward_current_func = custom_backward_current_func

        self.cell_num = cell_num

        # Constants
        self.k = 1.3806503 * 10 ** -23
        self.q = 1.60217646 * 10 ** -19

        # STC parameters
        self.Gstc = Gstc
        self.Tstc = Tstc + 273

        # Environmental conditions
        self.G = G
        self.T = T + 273

        # Experimental data
        self.e_current = experimental_I
        self.e_voltage = experimental_V

        # Solar cell parameters
        self.Voc = Voc
        self.Isc = Isc
        self.Pmpp = Pmpp
        self.Impp = Impp
        self.Vmpp = Vmpp

        self.Ku = Ku
        self.Ki = Ki
        self.Kp = Kp
        self.dT = self.T - self.Tstc

        # Model parameters
        self.I0 = I0
        self.Ipv = Ipv
        self.start_slope = start_slope
        self.end_slope = end_slope
        self.Rs = None
        self.Rp = None
        self.Vt = self.find_thermal_voltage() if Vt is None else Vt

        # Impurity coefficients
        self.a1 = a1
        self.a2 = a2

        """if self.Vt is None:
            self.Vt = self.find_thermal_voltage()
        self.Ipv_estimate = self.find_photovoltaic_current_estimate()
        self.I0_estimate = self.find_backward_current_estimate()
        """

        # Algorithm's variables
        self.is_experimental_data_provided = is_experimental_data_provided
        self.fixed_point_method_tolerance = fixed_point_method_tolerance
        self.mesh_points_num = mesh_points_num
        self.brute_force_steps = brute_force_steps
        self.brute_force_range = 0.3
        self.mode = mode
        self.eta = 0.2
        self.approx_range_minimization = floor(
            len(self.e_voltage) / 10) if approx_range_minimization is None else approx_range_minimization

        # Approximation
        self.I_0 = self.e_current[0]
        self.V_0 = self.e_voltage[-1]

        if is_experimental_data_provided:
            if 0 in self.e_voltage:
                self.voltage = self.e_voltage
            else:
                self.voltage = [0] + self.e_voltage
        else:
            self.create_voltage_mesh()

        self.start_fit_points_number = floor(len(self.voltage) * 0.22)
        self.end_fit_points_number = floor(len(self.voltage) * 0.085)

        self.area1_2 = int(0.55 * len(self.voltage))
        self.area2_3 = int(0.9 * len(self.voltage))

        if self.start_slope is None:
            self.start_slope = self.find_slope("start")
        if self.end_slope is None:
            self.end_slope = self.find_slope("end")

        self.beta = (exp(self.V_0 / self.Vt / self.a1) +
                     exp(self.V_0 / self.Vt / self.a2)) / \
                    (exp(self.V_0 / self.Vt / self.a1) +
                     exp(self.V_0 / self.Vt / a2) - 2)

        # Algorithm parameters
        self.current = None
        self.approx_error = None
        if self.Ipv is None:
            self.Ipv = self.find_photovoltaic_current()

        if self.I0 is None:
            self.Rs, self.Rp, self.I0 = self.find_parameters()

    def run(self, weight_arr=None):
        self.current, self.Rs, self.Rp, self.approx_error = self.find_best_fit(weight_arr)
        print("\nRs = ", self.Rs, "\nRp = ", self.Rp, "\nRS Error = ", self.approx_error)

    def recalc(self, G):
        self.G = G
        self.Ipv = self.find_photovoltaic_current()
        self.current = self.fixed_point_method(self.Rs, self.Rp)
        return self.current

    def create_voltage_mesh(self) -> Sequence[Union[float, int]]:
        mesh_center_index = self.mesh_points_num // 2
        return np.concatenate(
            (
                np.linspace(0, self.Vmpp, mesh_center_index, endpoint=False),
                np.linspace(self.Vmpp, self.Voc, self.mesh_points_num - mesh_center_index)
            )
        )

    def find_thermal_voltage(self) -> Union[float, int]:
        if self.custom_thermal_voltage_func is None:
            return self.cell_num * self.k * self.T / self.q
        else:
            return self.custom_thermal_voltage_func()

    def find_slope(self, slope_point='start') -> Union[float, int]:
        if slope_point == 'start':
            k, b = np.polyfit(self.voltage[:self.start_fit_points_number:],
                              self.e_current[:self.start_fit_points_number:], 1)
            return k
        if slope_point == 'end':
            a, b, c = np.polyfit(self.voltage[-self.end_fit_points_number::],
                                 self.e_current[-self.end_fit_points_number::], 2)
            return a * self.V_0 * 2 + b

    def find_photovoltaic_current(self) -> Union[float, int]:
        return (self.end_slope * self.start_slope * self.I_0 * self.Vt) / (
                (self.start_slope * self.Vt - self.beta * self.I_0 -
                 self.beta * self.V_0 * self.start_slope) *
                (self.start_slope - self.end_slope)) - \
            self.end_slope * self.I_0 / (self.start_slope - self.end_slope) * self.G / self.Gstc

    def find_parameters(self):
        Isc_g = self.I_0 * self.G / self.Gstc
        Rs = (Isc_g - self.Ipv) / self.Ipv / self.start_slope
        Rp = -Isc_g / self.start_slope / self.Ipv
        I0 = (self.Ipv - self.V_0 / Rp) / \
             (exp(self.V_0 / self.Vt / self.a1) + exp(self.V_0 / self.Vt / self.a2) - 2)
        return Rs, Rp, I0

    def fixed_point_method(self, Rs, Rp):
        current = [self.Ipv - Rs * self.Ipv / Rp]
        voltage = self.voltage[1::] if 0 in self.voltage else self.voltage
        lIb = log(self.I0)

        for V in voltage:
            prev_current = current[-1]
            while True:
                arg = (V + prev_current * Rs) / self.Vt
                new_current = self.Ipv - (V + prev_current * Rs) / Rp - exp(arg + lIb) - \
                              exp(arg / self.a2 + lIb) + 2 * self.I0
                prev_current = prev_current * (1 - self.eta) + self.eta * new_current

                if abs(prev_current - new_current) < self.fixed_point_method_tolerance:
                    break
            current.append(prev_current)
        return current

    def find_mape(self, experimental_data, approximation):
        error = 0

        if len(experimental_data) == len(approximation):
            if self.mode == 'overall':
                for exper_val, approx_val in zip(experimental_data, approximation):
                    if exper_val == 0:
                        continue
                    error += abs(exper_val - approx_val) / abs(exper_val) * 100
                return error / len(approximation)

            else:
                for exper_val, approx_val in zip(experimental_data[self.area1_2:self.area2_3],
                                                 approximation[self.area1_2:self.area2_3]):
                    if exper_val == 0:
                        continue
                    error += abs(exper_val - approx_val) / abs(exper_val) * 100
                return error / self.approx_range_minimization / 2
        else:
            raise ValueError("Different arrays length")

    def find_rmse(self, experimental_data, approximation, weight=None):
        if self.mode == "overall":
            if len(experimental_data) == len(approximation):
                error = []
                for exper_val, approx_val in zip(experimental_data, approximation):
                    if exper_val == 0:
                        continue
                    error.append((exper_val - approx_val) ** 2)
                if weight is not None:
                    error = weight[1:] * np.array(error)
                    error = np.sum(error) / np.sum(weight)
                else:
                    error = np.sum(error)
                return np.sqrt(error / len(approximation))
            else:
                print(len(approximation))
                print(len(experimental_data))
                raise ValueError("Different arrays length")

        else:
            if len(experimental_data) == len(approximation):
                error = 0
                for exper_val, approx_val in zip(experimental_data[self.area1_2:self.area2_3],
                                                 approximation[self.area1_2:self.area2_3]):
                    if exper_val == 0:
                        continue
                    error += abs(exper_val - approx_val) / abs(exper_val) * 100
                return error / self.approx_range_minimization / 2
            else:
                raise ValueError("Different arrays length")

    @staticmethod
    def progress_bar(idx, iter_num):
        idx += 1
        print('\rCompleted {}% |{:<20}|'.format(int(idx / iter_num * 100),
                                                '█' * int(idx / iter_num * 20)), end='')

    def find_best_fit(self, weight_arr=None):
        c = self.brute_force_range
        best_fit_error = 100
        best_fit_current = None
        best_fit_rs = None
        best_fit_rp = None

        for idx, Rs in enumerate(np.linspace(self.Rs * (1 - c), self.Rs * (1 + c),
                                             self.brute_force_steps)):
            self.progress_bar(idx, self.brute_force_steps)
            for Rp in np.linspace(self.Rp * (1 - c), self.Rp * (1 + c), self.brute_force_steps):
                current = self.fixed_point_method(Rs, Rp)
                approx_error = self.find_rmse(self.e_current, current, weight_arr)

                if approx_error < best_fit_error:
                    best_fit_error = approx_error
                    best_fit_current = current
                    best_fit_rs = Rs
                    best_fit_rp = Rp

        self.approx_error = best_fit_error
        self.current = best_fit_current
        self.Rs = best_fit_rs
        self.Rp = best_fit_rp
        return best_fit_current, best_fit_rs, best_fit_rp, best_fit_error

    @staticmethod
    def find_power(voltage, current):
        return [V * I for V, I in zip(voltage, current)]

    @staticmethod
    def find_max_power_index(power):
        max_power_index = 0
        max_power = 0
        for index, power in enumerate(power):
            if power > max_power:
                max_power_index = index
                max_power = power
        return max_power_index, max_power


def get_data(file):
    x_coord, y_coord = [], []
    for line in file:
        buff = list(map(float, line.strip(" ").split()))
        x_coord.append(buff[0])
        y_coord.append(buff[1])
    return x_coord, y_coord


def get_data_1(file):
    x_coord, y_coord = [], []
    for line in file:
        buff = list(map(float, line.strip(" ").split(", ")))
        x_coord.append(buff[0])
        y_coord.append(buff[1])
    return x_coord, y_coord


def normalize(arr, zero_padding=False, offset_coef=0.001):
    min_val = np.min(arr)
    max_val = np.max(arr)

    if zero_padding:
        new_arr = (arr - min_val) / (max_val - min_val)
    else:
        offset = offset_coef * abs(min_val)
        new_arr = (arr - min_val + offset) / (max_val - min_val + offset)
    return new_arr

In [ ]:
rs = []
rp = []

In [ ]:
r_file = open("C:\\Users\\Martyniuk Vadym\\Desktop\\solar_panel_modeling\\input_files\\" +
                  "KC200GT_new.txt")
voltage, current = get_data(r_file)
voltage = np.array(voltage)
r_file.close()

KC200GT = SolarCell_new(cell_num=54, Voc=32.9, Isc=8.21, a1=1.0, a2=1.2,
                                thermal_coefficient_type='absolute', G=1000,
                                experimental_V=voltage,
                                experimental_I=current,
                                is_experimental_data_provided=True,
                                mode='overall')

power = KC200GT.find_power(KC200GT.e_voltage, KC200GT.e_current)
weight = normalize(power) ** 4
KC200GT.run(weight)

rs.append(KC200GT.Rs)
rp.append(KC200GT.Rp)

plt.plot(voltage, current)
plt.plot(KC200GT.voltage, KC200GT.current)
plt.show()

In [ ]:
r_file = open("C:\\Users\\Martyniuk Vadym\\Desktop\\" + "800.txt")
voltage, current = get_data_1(r_file)
voltage = np.array(voltage)
r_file.close()

KC200GT = SolarCell_new(cell_num=54, Voc=32.9, Isc=8.21, a1=1.0, a2=1.2,
                                thermal_coefficient_type='absolute', G=1000,
                                experimental_V=voltage,
                                experimental_I=current,
                                is_experimental_data_provided=True,
                                mode='overall')

power = KC200GT.find_power(KC200GT.e_voltage, KC200GT.e_current)
weight = normalize(power) ** 4
KC200GT.run(weight)

rs.append(KC200GT.Rs)
rp.append(KC200GT.Rp)

plt.plot(voltage, current)
plt.plot(KC200GT.voltage, KC200GT.current)
plt.plot(voltage, weight)
plt.grid()
plt.show()

In [ ]:
r_file = open("C:\\Users\\Martyniuk Vadym\\Desktop\\" + "600.txt")
voltage, current = get_data_1(r_file)
voltage = np.array(voltage)
r_file.close()

KC200GT = SolarCell_new(cell_num=54, Voc=32.9, Isc=8.21, a1=1.0, a2=1.2,
                                thermal_coefficient_type='absolute', G=1000,
                                experimental_V=voltage,
                                experimental_I=current,
                                is_experimental_data_provided=True,
                                mode='overall')

power = KC200GT.find_power(KC200GT.e_voltage, KC200GT.e_current)
weight = normalize(power) ** 4
KC200GT.run(weight)

rs.append(KC200GT.Rs)
rp.append(KC200GT.Rp)

plt.plot(voltage, current)
plt.plot(KC200GT.voltage, KC200GT.current)
plt.plot(voltage, weight)
plt.show()

In [ ]:
r_file = open("C:\\Users\\Martyniuk Vadym\\Desktop\\" + "400.txt")
voltage, current = get_data_1(r_file)
voltage = np.array(voltage)
r_file.close()

KC200GT = SolarCell_new(cell_num=54, Voc=32.9, Isc=8.21, a1=1.0, a2=1.2,
                                thermal_coefficient_type='absolute', G=1000,
                                experimental_V=voltage,
                                experimental_I=current,
                                is_experimental_data_provided=True,
                                mode='overall')

power = KC200GT.find_power(KC200GT.e_voltage, KC200GT.e_current)
weight = normalize(power) ** 4
KC200GT.run(weight)

rs.append(KC200GT.Rs)
rp.append(KC200GT.Rp)

plt.plot(voltage, current)
plt.plot(KC200GT.voltage, KC200GT.current)
plt.show()

In [ ]:
G = [1000, 800, 600, 400]

In [ ]:
plt.plot(G, rp)